Steane Code FTb encoding and detection
================================= 

1. Set up fault tolerant logical zero using method b for Steane code based on the parity matrix in the book by Nielsen MA, Chuang IL. Quantum Computation and Quantum Information, 10th Anniversary Edition. Cambridge University Press; 2016. p. 474

2. Set up fault tolerant ancilla as also described in Nielsen and Chuang.

3. Carry out the error correction in software. 
4. Not currently feasible as needs 2* (7 + 6 * 4) = 62 qubits

In [1]:
from qiskit import(
    QuantumCircuit,
    QuantumRegister,
    ClassicalRegister,
    execute,
    Aer
    )

from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

from circuits import SteaneCodeLogicalQubit

from helper_functions import (
    get_noise, 
    correct_qubit, 
    string_reverse, 
    process_FT_results
    )

In [2]:
SINGLE_GATE_ERRORS = ['x', 'y', 'z', 'h', 's', 'sdg']
TWO_GATE_ERRORS = ['cx', 'cz']

QUBIT_LIST = [2,4,5]

ANC_ZERO = '0000'
ANC_ONE = '0001'
ANCILLA_TYPES = 2
ANCILLA_QUBITS = 3
ANCILLA_MEASUREMENT_REPEATS = 3
DATA_MEASUREMENT_REPEATS = 3
DATA_MEAS_QUBITS_B = 7
DATA_MEAS_START = ANCILLA_TYPES * ANCILLA_QUBITS * ANCILLA_MEASUREMENT_REPEATS

DATA_MEAS_QUBITS = 1
DATA_START_B = DATA_MEAS_START + (DATA_MEAS_QUBITS_B * DATA_MEASUREMENT_REPEATS)

TESTING = False                          #Test by applying and X or Z error to one of the qubits
TEST_X_QUBIT = 0                         #Qbit to set bit flip- Starts at zero.
TEST_X_LOGICAL = 0                       #logical Qbit to set bit flip- Starts at zero.
TEST_Z_QUBIT = 0                         #Qbit to phase flip- Starts at zero.
TEST_Z_LOGICAL = 0                       #logical Qbit to set phase flip- Starts at zero.

NOISE = False                            #Test with noise
SHOTS = 1000                             #Number of shots to run    
MEASURE_NOISE = 0.0046                   #Measurement noise not relevant
SINGLE_GATE_DEPOLARISING = 0.000366      #Single gate noise   
TWO_GATE_DEPOLARISING = 0.0204
FACTOR = 1
SIMULATOR = Aer.get_backend('qasm_simulator')
SPACE = ' '
DATA_QUBITS = 7

In [3]:
parity_check_matrix = ['0001111',
                       '0110011',
                       '1010101'
                      ]

In [4]:
codewords = ['0000000',
             '1010101',
             '0110011',
             '1100110',
             '0001111',
             '1011010',
             '0111100',
             '1101001'
            ]

In [5]:
if NOISE:
    noise_model = get_noise(MEASURE_NOISE * FACTOR, SINGLE_GATE_DEPOLARISING * FACTOR, 
                            TWO_GATE_DEPOLARISING * FACTOR, SINGLE_GATE_ERRORS, 
                            TWO_GATE_ERRORS )

In [6]:
qubit = SteaneCodeLogicalQubit(2, parity_check_matrix, codewords, 
                                   ancilla = True, fault_tolerant_b = True, 
                                   fault_tolerant_ancilla = True, ancilla_rounds = 3,
                                   data_rounds = 3
                                  )
qubit.set_up_logical_zero(0)
for i in range(3):
    qubit.barrier()
    qubit.set_up_logical_zero(1)
    qubit.barrier()
    qubit.logical_gate_CX(0, 1)
    qubit.barrier()
    qubit.logical_measure_data_FT(logical_qubit = 1, measure_round = i + 1)
    qubit.barrier() 
qubit.logical_measure_data(0)
if TESTING:
    qubit.force_X_error(TEST_X_QUBIT, TEST_X_LOGICAL)   #force X error for testing
    qubit.barrier()
for i in range(ANCILLA_MEASUREMENT_REPEATS):
    qubit.set_up_ancilla(0)
    qubit.logical_measure_ancilla(0, i)
qubit.logical_measure_data(0)
if NOISE:
    result = execute(qubit, SIMULATOR, noise_model=noise_model, shots=SHOTS).result()
else:
    result = execute(qubit, SIMULATOR, shots=SHOTS).result()
counts = result.get_counts(qubit)
#print ('The different states can be counted.  The simulated result are', counts)

ValueError: Can't set up fault tolerant ancilla with two logical qubits due to memory size restrictions

In [ ]:
error_rate, rejected, accepted, valid, invalid = process_FT_results(counts, codewords, 
                                                                    anc_zero = ANC_ZERO,
                                                                    anc_one = ANC_ONE,
                                                                    verbose = True,  
                                                                    data_meas_start = DATA_MEAS_START, 
                                                                    data_start = DATA_START_B,
                                                                    ancilla_qubits = ANCILLA_QUBITS, 
                                                                    ancilla_meas_repeats = ANCILLA_MEASUREMENT_REPEATS,
                                                                    data_meas_qubits = DATA_MEAS_QUBITS, 
                                                                    data_meas_repeats = DATA_MEASUREMENT_REPEATS
                                                                    )
print(f'The error rate is {error_rate:.4f} ')

In [ ]:
qubit.draw(output='mpl', filename = 'Steane_code_FTb_encoding_and_detection_circuit.jpg', fold = 80)